<a href="https://colab.research.google.com/github/kake01/Dialogue/blob/master/%E5%90%8D%E5%8F%A4%E5%B1%8B%E5%A4%A7%E5%AD%A6%E3%81%AE%E5%89%8D%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#名古屋大コーパスの前処理,ベクトル化

## 環境構築

In [0]:
# janomeのインストール
pip install janome

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import urllib.request
import zipfile
import pickle
from janome.tokenizer import Tokenizer

## データセットの解凍と前処理と保存

###データの解凍

In [0]:
with zipfile.ZipFile('data.zip') as existing_zip:
  existing_zip.extractall('data')

###データの前処理

In [0]:
text = ""

# s = "私は柴犬【しばいぬ】とシャム猫【しゃむねこ】が大好きです。"
# s = re.sub("【[^】]+】", "", s)  # 【と】の間に】以外の文字が複数ある箇所を、空の文字列に置き換える
for data_num in range(129):
  with open("data/nucc/data_"+str(data_num+1)+".txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    texts = f.read()
  texts = re.sub("《[^》]+》", "", texts)  # ルビの削除
  texts = re.sub("［[^］]+］", "", texts)  # 読みの注意の削除
  texts = re.sub("〔[^〕]+〕", "", texts)  # 読みの注意の削除

# オリジナル部分
  texts = re.sub("＜[^＞]+＞", "", texts)  # ＜笑い＞の削除
  texts = re.sub("＠[^\n]+\n", "", texts)  # @から始まる文を削除

  texts = re.sub("F[^：]+：", "", texts)   # 誰が喋ったかの名前削除
  texts = re.sub("M[^：]+：", "", texts)   # 誰が喋ったかの名前削除

  texts = re.sub("[ 　\n「」『』（）｜※＊…]", "", texts)  # 全角半角スペース、改行、その他記号の削除
  text += texts

print("文字数:", len(text))
print(text[:90])

###データの保存

####.txtとして保存


In [0]:
# コーパスとして保存する
with open("corpus.txt", mode="w", encoding="utf-8") as f:
    f.write(text)

####.pickleとして保存

In [0]:
# ファイルの読み込み
with open("corpus.txt", mode="r", encoding="utf-8") as f:
  text = f.read()

seperator = "。"  # 。をセパレータに指定
sentence_list = text.split(seperator)  # セパレーターを使って文章をリストに分割する
sentence_list.pop() # 最後の要素は空の文字列になるので、削除
sentence_list = [x+seperator for x in sentence_list]  # 文章の最後に。を追加



t = Tokenizer()

words = []
for sentence in sentence_list:
    words.append(t.tokenize(sentence, wakati=True))   # 文章ごとに単語に分割し、リストに格納
    
with open('words.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(words, f)

In [0]:
with open('words.pickle', mode='rb') as f:
    wagahai_words = pickle.load(f)
print(wagahai_words[:500])

##単語のベクトル化(word2vec)

In [0]:
from gensim.models import word2vec

# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(wagahai_words,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=20,
                          sg = 0)

In [0]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現

In [0]:
print(model.wv.most_similar("怖い"))  # 最も似ている単語

In [0]:
# # モデルを保存
# model.save("model.model")
# # モデルを読み込む
# model = word2vec.Word2Vec.load("model.model")

with open('model.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(model.wv.vectors, f)